In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [2]:
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

In [3]:
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [4]:
# Additional imports for loading PDF documents and QA chain.
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [5]:
# Additional imports for loading Wikipedia content and QA chain.
from langchain_community.document_loaders import WikipediaLoader
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# Create the PDF loader and load the document.
#pdf_loader = PyPDFLoader('https://static.bc-edx.com/ai/ail-v-1-0/m22/lesson_1/datasets/Resume.pdf')
#documents = pdf_loader.load()

# Initialize the model.
#llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the QA chain using the LLM.
#chain = load_qa_chain(llm)

# Define a query as a string.
#query = 'Could I write to Jane Doe in Spanish and expect her to understand?'

# Pass the documents and the query to the chain, and print the result.
#result = chain.invoke({"input_documents": documents, "question": query})
#print(result["output_text"])

In [6]:
#Setting the Model

# Initialize the model with your OpenAI API key
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = 'gpt-3.5-turbo'  # We're using GPT 3.5 turbo as we don't really need GPT 4 for this as the info doesn't change that much. It provides what we need at a lower cost.
#Instantiating the llm we'll use and the arguments to pass
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Define the wikipedia topic as a string.
wiki_topic = "diabetes"

# Load the wikipedia results as documents, using a max of 2.
#included error handling- unable to load documents
try:
    documents = WikipediaLoader(query=wiki_topic, load_max_docs=2, load_all_available_meta=True).load()
except Exception as e:
    print("Failed to load documents:", str(e))
    documents = []
# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

In [7]:

#Define the function to call the LLM
def handle_query(user_query):
    if not documents:
        return "Source not loading info; please try again later."

    if user_query.lower() == 'quit':
        return "Goodbye!"

    try:
        # Pass the documents and the user's query to the chain, and return the result.
        result = chain.invoke({"input_documents": documents, "question": user_query})
        if result["output_text"].strip():
            return result["output_text"]
        else:
            return "I'm sorry. I'm not finding an good answer for you. Please rephrase your question or try another question."
    except Exception as e:
        return "An error occurred while searching for the answer: " + str(e)

# Create a Gradio interface
iface = gr.Interface(
    fn=handle_query,  # Callback function
    inputs=gr.Textbox(lines=2, placeholder="What is your question about diabetes?"),
    outputs=gr.Textbox(),
    title="Diabetes Information Query",
    description="Enter your question about diabetes and get information from Wikipedia."
)

# Run the Gradio app
iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
#Define the function to call the LLM

# Initial empty history
initial_history = []
def handle_query(user_query, history):
    if not documents:
        return "Source not loading info; please try again later.", history

    if user_query.lower() == 'quit':
        return "Goodbye!"

    try:
        # Pass the documents and the user's query to the chain, and return the result.
        result = chain.invoke({"input_documents": documents, "question": user_query})
        ai_response = result["output_text"].strip() if result ["output_text"].strip() else "I'm sorry. No good answer found.  Please try another question."
        
        #update the history with this round
        updated_history = history +[(user_query,ai_response)]
        return ai_response, updated_history
    except Exception as e:
        return f"An error occurred :{str(e)} " , history

# Create a Gradio interface
iface = gr.Interface(
    fn=handle_query,  # Callback function
    inputs=gr.Textbox(lines=2, placeholder="What is your question about diabetes?"),
    outputs=[gr.Textbox(), gr.State()]
    title="Diabetes Information Query",
    description="Enter your question about diabetes and get information from Wikipedia."
)

# Run the Gradio app
iface.launch(state=initial_history)

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1336, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/utils.py", line 726, in async_wrapper
    response = await f(*args, **kw

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()

/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Traceback (most recent call last):
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sonjambaro/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 13

In [ ]:
# Create a savings interest calculator function that takes 
# the balance, apr, and number of days, and returns the interest for the number of days. 
def calculate_interest(balance, apr, days):
    """
    Calculate the interest earned on a balance based on the annual percentage rate (APR) over a specified number of days.

    Parameters:
    - balance (float): The initial balance or principal amount.
    - apr (float): The Annual Percentage Rate (APR) as a percentage.
    - days (int): The number of days for which interest is calculated.

    Returns:
    float: The interest earned rounded to 2 decimal places.
    """
    apr_decimal = apr/100
    interest_rate = apr_decimal * (days/365)
    interest_earned = balance * interest_rate
    return round(interest_earned,2)

# Create an instance of the Gradio Interface application function with the following parameters. 
app = gr.Interface(fn=calculate_interest, 
                   inputs=["number","number", 'number'], outputs="number")
# Launch the app
app.launch()